# Title: Interactive Machine Leanring for Atmospheric data

Description: Allows students to select ML models, tune hyperparameters, and visualize predictions interactively.

In [ ]:
# =======================
# 1. Import Required Libraries
# =======================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import ipywidgets as widgets
from IPython.display import display

In [ ]:
# =======================
# 2. Generate Synthetic Atmospheric Data
# =======================
np.random.seed(42)
n_samples = 200

temperature = np.random.normal(25, 3, n_samples)
humidity = np.random.uniform(40, 80, n_samples)
no2 = np.random.normal(30, 5, n_samples)
ozone = 0.4*temperature - 0.3*humidity + 0.5*no2 + np.random.normal(0, 2, n_samples)

df = pd.DataFrame({'Temperature': temperature, 'Humidity': humidity, 'NO2': no2, 'Ozone': ozone})

X = df[['Temperature','Humidity','NO2']]
y = df['Ozone']

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# =======================
# 3. Define Interactive Function with Cross-Validation & Monte Carlo
# =======================
def train_evaluate_uncertainty(model_name, rf_estimators, svr_c, svr_gamma, n_mc_sim):
    if model_name == 'Linear Regression':
        model = LinearRegression()
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
        feature_importance = None
    elif model_name == 'Random Forest':
        model = RandomForestRegressor(n_estimators=rf_estimators, random_state=42)
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
        feature_importance = model.feature_importances_
    elif model_name == 'SVM':
        model = SVR(kernel='rbf', C=svr_c, gamma=svr_gamma)
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
        feature_importance = None
    else:
        print("Invalid model selected.")
        return
    
    # Metrics
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    
    # Cross-validation R²
    cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='r2')
    
    # Monte Carlo prediction intervals
    mc_predictions = []
    for _ in range(n_mc_sim):
        X_perturbed = X_test_scaled + np.random.normal(0, 0.05, X_test_scaled.shape)
        mc_predictions.append(model.predict(X_perturbed))
    mc_predictions = np.array(mc_predictions)
    y_mc_mean = mc_predictions.mean(axis=0)
    y_mc_lower = np.percentile(mc_predictions, 2.5, axis=0)
    y_mc_upper = np.percentile(mc_predictions, 97.5, axis=0)
    
    # Display metrics
    print(f"Model: {model_name}")
    print(f"RMSE: {rmse:.3f}, MAE: {mae:.3f}, R²: {r2:.3f}")
    print(f"Cross-validated R² scores: {cv_scores}")
    print(f"Mean CV R²: {cv_scores.mean():.3f}")
    
    # Plot observed vs predicted with Monte Carlo uncertainty
    plt.figure(figsize=(8,6))
    plt.errorbar(range(len(y_test)), y_mc_mean, yerr=[y_mc_mean-y_mc_lower, y_mc_upper-y_mc_mean],
                 fmt='o', label='Predicted ± MC interval', color='green', alpha=0.7)
    plt.scatter(range(len(y_test)), y_test, label='Observed', color='blue')
    plt.xlabel('Test Sample Index')
    plt.ylabel('Ozone')
    plt.title('Observed vs Predicted Ozone with Monte Carlo Uncertainty')
    plt.legend()
    plt.show()
    
    # Feature importance plot for Random Forest
    if feature_importance is not None:
        plt.figure(figsize=(6,4))
        plt.bar(X.columns, feature_importance)
        plt.xlabel('Feature')
        plt.ylabel('Importance')
        plt.title('Random Forest Feature Importance')
        plt.show()


In [ ]:
# =======================
# 4. Create Interactive Widgets
# =======================
model_selector = widgets.Dropdown(
    options=['Linear Regression', 'Random Forest', 'SVM'],
    value='Linear Regression',
    description='Model:'
)

rf_estimators_slider = widgets.IntSlider(
    value=100, min=10, max=500, step=10, description='RF n_estimators:'
)

svr_c_slider = widgets.FloatLogSlider(
    value=1.0, base=10, min=-1, max=3, step=0.1, description='SVM C:'
)

svr_gamma_slider = widgets.FloatLogSlider(
    value=0.1, base=10, min=-3, max=1, step=0.1, description='SVM gamma:'
)

mc_sim_slider = widgets.IntSlider(
    value=100, min=10, max=1000, step=10, description='MC simulations:'
)

ui = widgets.VBox([
    model_selector,
    rf_estimators_slider,
    svr_c_slider,
    svr_gamma_slider,
    mc_sim_slider
])

out = widgets.interactive_output(
    train_evaluate_uncertainty,
    {'model_name': model_selector,
     'rf_estimators': rf_estimators_slider,
     'svr_c': svr_c_slider,
     'svr_gamma': svr_gamma_slider,
     'n_mc_sim': mc_sim_slider}
)

display(ui, out)